In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

Using TensorFlow backend.


In [ ]:
model = ResNet50(weights='imagenet')

In [3]:
img_path = './husky.jpeg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

Predicted: [('n02109961', 'Eskimo_dog', 0.56537235), ('n02110185', 'Siberian_husky', 0.4156282), ('n02110063', 'malamute', 0.015444505)]


In [4]:
## Keras will download pretrained model to ~/.keras/models
!ls ~/.keras/models

imagenet_class_index.json  resnet50_weights_tf_dim_ordering_tf_kernels.h5


## SageMaker Neo

#### You can't use ~/.keras/models/resnet50_weights_tf_dim_ordering_tf_kernels.h5 into neo. It can't generate the neo model. You must save current model with imagenet weights

In [5]:
## Save model with imagenet weights
## You can't use ~/.keras/models/resnet50_weights_tf_dim_ordering_tf_kernels.h5 into neo. It will be failed.
model.save('./keras-resnet50-imagenet.h5')

## upload to s3
!tar cvfz ./keras-resnet50-imagenet.tar.gz ./keras-resnet50-imagenet.h5
!aws s3 cp keras-resnet50-imagenet.tar.gz s3://beyoung-sagemaker-us-east-1/neo/keras-resnet50-imagenet.tar.gz 

./keras-resnet50-imagenet.h5
upload: ./keras-resnet50-imagenet.tar.gz to s3://beyoung-sagemaker-us-east-1/neo/keras-resnet50-imagenet.tar.gz


In [8]:
## Get sagemaker execution role
from sagemaker import get_execution_role
role = get_execution_role()

In [ ]:
## Create neo job
## neo trouble shooting https://docs.aws.amazon.com/sagemaker/latest/dg/neo-troubleshooting.html
## target devices: 'lambda'|'ml_m4'|'ml_m5'|'ml_c4'|'ml_c5'|'ml_p2'|'ml_p3'|'ml_inf1'|'jetson_tx1'|'jetson_tx2'|'jetson_nano'|'rasp3b'|'deeplens'|'rk3399'|'rk3288'|'aisage'|'sbe_c'|'qcs605'|'qcs603'

import boto3
import time
client = boto3.client('sagemaker')
job_name_prefix = 'neo-resnet' 
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime()) 
job_name = job_name_prefix + timestamp
data_input_config = '{"input_1":[1,3,224,224]}'


response = client.create_compilation_job(
    CompilationJobName=job_name,
    RoleArn=role,
    InputConfig={
        'S3Uri': 's3://beyoung-sagemaker-us-east-1/neo/keras-resnet50-imagenet.tar.gz',
        'DataInputConfig': data_input_config,
        'Framework': 'KERAS'
    },
    OutputConfig={
        'S3OutputLocation': 's3://beyoung-sagemaker-us-east-1/neo/output',
        'TargetDevice': 'deeplens'
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 300,
        'MaxWaitTimeInSeconds': 300
    }
)

response